In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import urllib
import csv
import glob
import argparse
import numpy as np
import pandas as pd
from pandas import concat
from pandas import DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn import datasets, linear_model
import scipy.io as sio
import datetime
import requests
from dateutil.relativedelta import relativedelta
import contextily as ctx
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime, timezone
from mintpy.utils import readfile, writefile
from mintpy.defaults.plot import *
from mintpy.objects.gps import search_gps, GPS
from mintpy.objects import sensor
from mintpy.utils import utils as ut
from mintpy.view import prep_slice, plot_slice
from mintpy.cli import view, timeseries2velocity

from utils.helper_functions import prepend_scratchdir_if_needed
from utils.helper_functions import find_nearest_start_end_date
from utils.plot_functions import get_basemap

from utils.helper_functions import find_nearest_start_end_date
from utils.helper_functions import get_dem_extent
from utils.plot_functions import get_basemap
from utils.plot_functions import plot_shaded_relief
from utils.plot_functions import modify_colormap
from utils.plot_functions import add_colorbar
from utils.seismicity import get_earthquakes
from utils.seismicity import normalize_earthquake_times
from utils.gps import get_GPS
from utils.insar import get_eos_file
from utils.insar import generate_view_velocity_cmd
from utils.insar import generate_view_ifgram_cmd

%load_ext jupyter_ai

In [ ]:
# set data directories and plot parameters
# Need to fix (introduce file):
# work_dir = "MaunaLoaSenAT124/mintpy/S1_IW23_124_0059_0063_20180905_XXXXXXXX_N19122_N19785_W155940_W155223.he5"
# work_dir = os.getenv('SCRATCHDIR') + '/MaunaLoaSenAT124/mintpy/S1_IW23_124_0059_0063_20180905_XXXXXXXX_N19122_N19785_W155940_W155223.he5'

work_dir = "MaunaLoaSenAT124"

plot_box =[19.29, 19.6, -155.79, -155.41]
depth_range="0 10"
cmap_name = "plasma_r"; exclude_beginning = 0.2; exclude_end = 0.2
work_dir, start_date, end_date, scale_fac, key_length, unit ="MaunaLoaSenDT87","20210101", "20210401", 500, 4, "cm"    # Desc

plot_type = 'shaded_relief' 
plot_type = 'ifgram' 
plot_type = 'velocity' 


In [ ]:
EXAMPLE = """example:
  plot_data.py  MaunaLoaSenDT87 MaunaLoaSenAT124 
  plot_data.py  MaunaLoaSenDT87       
"""

def create_parser(subparsers=None):
    synopsis = 'Plotting of InSAR, GPS and Seismicity data'
    epilog = EXAMPLE
    parser = argparse.ArgumentParser(description='Plot InSAR, GPS and seismicity data\n')

    # parser.add_argument('data_dir', nargs=1, help='Directory(s) with InSAR data.\n')
    parser.add_argument('data_dir', nargs='*', help='Directory(s) with InSAR data.\n')

    parser.add_argument('--plotBox', '--subset-lalo', dest='plot_box', default='19.29:19.6,-155.79:-155.41',
                        help='flag to remove gps vectors')
    parser.add_argument('--startDate', dest='start_date', default='20220801',help='start date')
    parser.add_argument('--endDate', dest='end_date', default='20221115',help='end date')
    parser.add_argument('--seismicity', dest='flag_seismicity', action='store_true', default=True,
                        help='flag to add seismicity')
    parser.add_argument('--noseismicity', dest='flag_noseismicity', action='store_true',default=False,
                        help='flag to remove seismicity')
    parser.add_argument('--gps', dest='flag_gps', action='store_true', default=True,
                        help='flag to add gps vectors')
    parser.add_argument('--nogps', dest='flag_nogps', action='store_true',default=False,
                        help='flag to remove gps vectors')
    parser.add_argument('--plotType', dest='plot_type', default='velocity',
                        help='Type of plot: velocity, ifgram or shaded_relief (Default: velocity).')
    
    
    args = parser.parse_args()
    
    if len(args.data_dir) < 1 or len(args.data_dir) > 2:
        parser.error('ERROR: You must provide 1 or 2 directory paths.')
    if len(args.data_dir) == 2:
        args.plot_type='horzvert'
    if args.flag_noseismicity:
       args.flag_seismicity = False
    del args.flag_noseismicity
    if args.flag_nogps:
       args.flag_gps = False
    del args.flag_nogps
    
    inps = args
    inps.plot_box = [float(val) for val in inps.plot_box.replace(':', ',').split(',')]

    #inps.argv = iargs if iargs else sys.argv[1:]

    return inps

def is_jupyter():
    jn = True
    try:
        get_ipython()
    except:
        jn = Fals
    return jn

def main(iargs=None):
    print('iargs:', iargs)
    # inps = cmd_line_parse(iargs)
    inps = create_parser(iargs)

    print('inps:',inps)
    return inps

###########################################################################################

if __name__ == '__main__':

    if is_jupyter():
        cmd = 'test_parse.py --help'
        cmd = 'test_parse.py MaunaLoaSenDT87 --plotType ifgram --noseismicity --nogps'
        cmd = 'test_parse.py MaunaLoaSenAT124 MaunaLoaSenDT87'
        cmd = 'test_parse.py MaunaLoaSenAT124'

        cmd = cmd.rstrip()
        sys.argv = cmd.split(' ')

    inps=main(sys.argv[1:])
    

In [ ]:
# Hardwired for Hawaii
date12 = '20221216_20230202'  # to plot an example descending ifgram
GPS_dir = os.getenv('SCRATCHDIR') + '/MaunaLoa/MLtry/data/'
data_dir = GPS_dir
dem_file = data_dir + 'demGeo.h5'  
lines=sio.loadmat(data_dir + '/hawaii_lines_new.mat',squeeze_me=True);
GPS_list_file = GPS_dir + 'GPS_BenBrooks_03-05full.txt'

# get dem, earthquake and GPS data, normalize event times for plotting)
dem_shade, dem_extent = get_basemap(dem_file)
events_df = get_earthquakes(start_date, end_date, plot_box)
norm_times = normalize_earthquake_times(events_df, start_date, end_date)
cmap = modify_colormap( cmap_name = cmap_name, exclude_beginning = exclude_beginning, exclude_end = exclude_end, show = False)

gps,lon,lat,U,V,Z,quiver_label = get_GPS(GPS_dir, GPS_list_file, plot_box, start_date, end_date, unit, key_length)

In [ ]:
%%capture

work_dir = prepend_scratchdir_if_needed(work_dir)
if plot_type is 'velocity':
    eos_file, out_dir, vel_file = get_eos_file(work_dir)
    start_date_mod, end_date_mod = find_nearest_start_end_date(eos_file, start_date, end_date)
    start_date = start_date_mod
    end_date = end_date_mod
    cmd = f'{eos_file} --start-date {start_date} --end-date {end_date} --output {vel_file}'
    timeseries2velocity.main( cmd.split() )

In [ ]:
# Cell to plot GPS on shaded relief or interferogram
# %load_ext autoreload
# %autoreload 2
  
fig, ax = plt.subplots(figsize=[12, 5] )

if plot_type is 'shaded_relief':
    plot_shaded_relief(ax, dem_file, plot_box = plot_box)
else:
    if plot_type is 'velocity':
        cmd = generate_view_velocity_cmd(vel_file, plot_box)
    elif plot_type is 'ifgram':
        cmd = generate_view_ifgram_cmd(work_dir, date12, plot_box)
    data, atr, inps = prep_slice(cmd)
    ax, inps, im, cbar = plot_slice(ax, data, atr, inps)

# plot lines and events
ax.scatter(events_df["Longitude"],events_df["Latitude"],s=2*events_df["Magnitude"] ** 3,
         c=norm_times,cmap=cmap,alpha=0.8)
add_colorbar(ax = ax, cmap = cmap, start_date = start_date, end_date = end_date)
ax.set_title('Events:  ' + start_date + ' - ' + end_date);
quiv=ax.quiver(lon, lat, U, V, scale = scale_fac, color='blue')
ax.quiverkey(quiv, -155.50, 19.57, key_length*10 , quiver_label, labelpos='N',coordinates='data',
                  color='blue',fontproperties={'size': 20}) 

In [ ]:
# # print(eos_file)
# atr = readfile.read_attribute(eos_file)
# mask, dict_mask = readfile.read(eos_file, datasetName="/HDFEOS/GRIDS/timeseries/quality/mask")
# # temporal_coherence, dict_temp_coh = readfile.read(eos_file, datasetName="/HDFEOS/GRIDS/timeseries/quality/temporalCoherence")

# atr['FILE_TYPE'] = 'mask'
# mask_file = vel_file.replace('velocity','maskTempCoh')
# writefile.write(mask, out_file=mask_file, metadata=atr)

# fig, ax = plt.subplots(figsize=[12, 5])
# cmd = generate_view_velocity_cmd(vel_file, mask_file, plot_box)
# #cmd = generate_view_velocity2_cmd(vel_file, mask_file, plot_box)

# print(cmd)
# data, atr, inps = prep_slice(cmd)
# ax, inps, im, cbar = plot_slice(ax, data, atr, inps)